# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [14]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft, dataam, datanflx], axis=1, sort=False)
data = data[90:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']

up = .03
down =-.03
ydata = data['% Change']
ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""

for index, row in ydata.iterrows():
    temp = list(ydata['% Change'][index-90:index+13-90].values)
    ydata.set_value(index,'newy',temp)

    
print(ydata)




/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


      % Change                                     newy
90           0  [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0]
91           0  [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
92           0  [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
93           0  [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
94           2  [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
95           0  [0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
96           2  [2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2]
97           0  [0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0]
98           0  [0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0]
99           0  [0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0]
100          2  [2, 0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2]
101          0  [0, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2]
102          0  [0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0]
103          2  [2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0]
104          0  [0, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2]
105          0  [0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2, 2]
106          2  [2, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2